In [9]:
import pandas as pd
import numpy as np
import ta

In [ ]:
import requests
import pandas as pd
import feedparser

#Fetching of data from serveral sources and merging

oanda_api_key = "b9846701e3d5d9b2d73127d5040aac28-5d86be73abbdec141a31891cb532d340"
oanda_url = "https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles?granularity=M30"
headers = {"Authorization": f"Bearer {oanda_api_key}"}
response = requests.get(oanda_url, headers=headers).json()

forex_data = pd.DataFrame(response['candles'])


if 'mid' in forex_data.columns:
    forex_data['open'] = forex_data['mid'].apply(lambda x: float(x['o']))
    forex_data['high'] = forex_data['mid'].apply(lambda x: float(x['h']))
    forex_data['low'] = forex_data['mid'].apply(lambda x: float(x['l']))
    forex_data['close'] = forex_data['mid'].apply(lambda x: float(x['c']))
    forex_data.drop(columns=['mid'], inplace=True)
else:
    print("'mid' column missing from forex_data!")


forex_data['time'] = pd.to_datetime(forex_data['time']).dt.tz_localize(None)


forex_data['date_time'] = forex_data['time'].dt.floor('D')  
forex_data['rounded_time'] = forex_data['time'].dt.floor('h')  

fred_api_key = "5de83463f5e12c01d110a2322a075ecc"
fred_series = {
    "inflation": "CPIAUCSL",
    "gdp": "GDPC1",
    "unemployment": "UNRATE",
    "interest_rate": "DGS10"
}

def get_fred_data(series_id):
    url = f"https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={fred_api_key}&file_type=json"
    response = requests.get(url).json()

    if 'observations' not in response:
        print(f"Failed to fetch {series_id}")
        return pd.DataFrame()
    
    data = pd.DataFrame(response['observations'])
    data['date'] = pd.to_datetime(data['date'])
    data[series_id] = pd.to_numeric(data['value'], errors='coerce')
    return data[['date', series_id]]


fred_data = pd.DataFrame()
for label, series_id in fred_series.items():
    indicator_df = get_fred_data(series_id)
    if indicator_df.empty:
        continue
    if fred_data.empty:
        fred_data = indicator_df
    else:
        fred_data = fred_data.merge(indicator_df, on='date', how='outer')

fred_data.sort_values('date', inplace=True)
fred_data.ffill(inplace=True) 


forex_data.sort_values('date_time', inplace=True)
merged_data = pd.merge_asof(
    forex_data,
    fred_data,
    left_on='date_time',
    right_on='date',
    direction='backward'
)

feed_url = "https://feeds.bbci.co.uk/news/business/rss.xml"
feed = feedparser.parse(feed_url)

news_df = pd.DataFrame({
    'headline': [entry.title for entry in feed.entries],
    'datetime': pd.to_datetime([entry.published for entry in feed.entries], errors='coerce')
})

keywords = ['interest rate', 'inflation', 'central bank', 'monetary policy']
news_df = news_df[news_df['headline'].str.contains('|'.join(keywords), case=False, na=False)]

news_df['rounded_time'] = news_df['datetime'].dt.floor('h')  
news_grouped = news_df.groupby('rounded_time').agg({'headline': lambda x: ' | '.join(x)}).reset_index()
merged_data = merged_data.merge(news_grouped, on='rounded_time', how='left')


merged_data = merged_data.drop(columns=['date_time','date','rounded_time'])

merged_data = merged_data.sort_values(by='time',ascending=False)
merged_data


,complete,volume,time,open,high,low,close,CPIAUCSL,GDPC1,UNRATE,DGS10,headline
499,False,714,2025-04-17 22:30:00,1.13680,1.13692,1.13673,1.13682,319.615,23542.349,4.2,4.29,NaN
478,True,2942,2025-04-17 22:00:00,1.13695,1.13721,1.13668,1.13682,319.615,23542.349,4.2,4.29,NaN
472,True,411,2025-04-17 21:30:00,1.13694,1.13700,1.13670,1.13694,319.615,23542.349,4.2,4.29,NaN
471,True,137,2025-04-17 21:00:00,1.13673,1.13698,1.13632,1.13694,319.615,23542.349,4.2,4.29,NaN
470,True,1654,2025-04-17 20:30:00,1.13693,1.13699,1.13623,1.13646,319.615,23542.349,4.2,4.29,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
16,True,9734,2025-04-03 15:00:00,1.11185,1.11190,1.10586,1.10614,319.615,23542.349,4.2,4.06,NaN
15,True,9738,2025-04-03 14:30:00,1.10950,1.11277,1.10935,1.11184,319.615,23542.349,4.2,4.06,NaN
14,True,16033,2025-04-03 14:00:00,1.10846,1.11046,1.10798,1.10950,319.615,23542.349,4.2,4.06,NaN
17,True,14177,2025-04-03 13:30:00,1.10942,1.11061,1.10702,1.10815,319.615,23542.349,4.2,4.06,NaN


In [ ]:



forex_data['sma_20'] = ta.trend.sma_indicator(forex_data['close'], window=20) 
forex_data['ema_20'] = ta.trend.ema_indicator(forex_data['close'], window=20) 

forex_data['macd'] = ta.trend.macd(forex_data['close'])  
forex_data['macd_signal'] = ta.trend.macd_signal(forex_data['close'])  
forex_data['macd_histogram'] = ta.trend.macd_diff(forex_data['close'])  

forex_data['rsi'] = ta.momentum.rsi(forex_data['close'], window=14)  

forex_data['bollinger_middle'] = ta.volatility.bollinger_mavg(forex_data['close'], window=20)  
forex_data['bollinger_upper'] = ta.volatility.bollinger_hband(forex_data['close'], window=20) 
forex_data['bollinger_lower'] = ta.volatility.bollinger_lband(forex_data['close'], window=20) 


forex_data['atr'] = ta.volatility.average_true_range(forex_data['high'], forex_data['low'], forex_data['close'], window=14)


forex_data['stochastic_k'] = ta.momentum.stoch(forex_data['high'], forex_data['low'], forex_data['close'], window=14)
forex_data['stochastic_d'] = ta.momentum.stoch_signal(forex_data['high'], forex_data['low'], forex_data['close'], window=14)


forex_data['obv'] = ta.volume.on_balance_volume(forex_data['close'], forex_data['volume'])


forex_data['roc'] = ta.momentum.roc(forex_data['close'], window=10)


forex_data['vroc'] = ta.momentum.roc(forex_data['volume'], window=10)


forex_data['price_change'] = forex_data['close'] - forex_data['open']


forex_data['price_pct_change'] = (forex_data['close'] - forex_data['open']) / forex_data['open'] * 100


forex_data['candle_range'] = forex_data['high'] - forex_data['low']


forex_data['body_size'] = abs(forex_data['close'] - forex_data['open'])


forex_data['upper_shadow'] = forex_data['high'] - pd.DataFrame([forex_data['close'], forex_data['open']]).max()


forex_data['lower_shadow'] = pd.DataFrame([forex_data['close'], forex_data['open']]).min() - forex_data['low']


forex_data['deviation_pts'] = forex_data['price_pct_change']  


In [ ]:
# for i in range(1, 4):
#     forex_data[f'return_lag_{i}'] = forex_data['Close'].pct_change(i)
# # forex_data['hour'] = forex_data['Time'].dt.hour
# # forex_data['day_of_week'] = forex_data.index.dayofweek

# # forex_data.dropna(inplace=True)
